# Cognitive Portfolio Optimization

## This notebook outlines steps to combine AI with Portfolio Optimization service

This notebook requires a tradable universe. The experimental version of the Portfolio Optimization service http://console.bluemix.net/catalog/services/portfolio-optimization has a static data set for users to play with; a dynamic data set will be integrated in a future release. Note that much of the data in this sample set is _representative_ of actual data. 

The spreadsheet "Universe Data" contains all relevant information on the current sample data set.

## Step 1 - Load in Data Set

Load the file into the notebook. Apparently there's a command that simply inserts the code into your notebook that converts the csv file to a pandas dataframe!

In [1]:
from io import StringIO
import requests
import json
import pandas as pd

In [2]:
universe_data = pd.read_csv('Universe Data.csv')
universe_data.head()

,Ticker,Name,CUSIP,Sector,Geography,Asset Class,Last Close Price,One Month Return,Risk Score,Controversy,...,Social,Sustainability,Has Tobacco,Has Alcohol,Has Gambling,Has Military,Has Fossil Fuels,Benchmark - Conservative,Benchmark - Moderate,Benchmark - Aggressive
0,CX_IE00BY7QL619_NYQ,JOHNSON CONTROLS INTERNATIONAL PLC,00BY7QL61,Industrials,Domestic,Equity,44.00,0.007803,5,Average,...,High,Low,0,0,0,0,0,0.0,0.0,0.0
1,CX_US02079KAA51_USD,ALPHABET INC,02079KAA5,Information Technology,Domestic,Corporate Bonds,105.39,-0.000429,3,Low,...,Low,Average,0,0,0,0,0,0.0,0.0,0.0
2,CX_US031162BG42_USD,AMGEN INC,031162BG4,Health Care,Domestic,Corporate Bonds,106.49,0.000237,4,Average,...,High,Low,0,0,0,0,0,0.0,0.0,0.0
3,CX_US031162BK53_USD,Amgen 5.15% 11/15/2041,031162BK5,Health Care,Domestic,Corporate Bonds,116.05,0.004569,3,High,...,High,Average,0,0,0,0,0,0.0,0.0,0.0
4,CX_US03232PAD06_USD,AmSurg Corp 5.625% 07/15/2022,03232PAD0,Health Care,Domestic,Corporate Bonds,104.00,0.000594,3,Average,...,Average,Average,0,0,0,0,0,0.0,0.0,0.0


## Step 2 - Get Investor Profile and Preferences as Natural Text

Watson Natural Language Understanding service http://console.bluemix.net/catalog/services/natural-language-understanding analyzes provided text (in text, url, or html format) for the specified semantic features.

It can extract entities, concepts, keywords, categories, relations, sentiment, and emotions from provided text.

Watson Language Classifier service also lets you build custom classifiers to classify texts provided that you have training data.

In [37]:
try:
    from watson_developer_cloud import NaturalLanguageUnderstandingV1 as NLU
    from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, ConceptsOptions, CategoriesOptions, EmotionOptions, RelationsOptions, SemanticRolesOptions
except:
    !pip install watson_developer_cloud

In [4]:
# Insert your username and password
apikey='sy3UugZgezzNAb8k_Ks42Eh_3VmE07wwE96m8Sxk8qXi'
nlu = NLU(
    iam_apikey=apikey,
    version='2018-03-16'
)

Create text (investor profile and preferences) to analyze.

In [5]:
investor_text='My name is John. I am 34. I work information technology sector. I would like to invest $30000 in my RRSP this year. I would like highly diversified portfolio. I do not like having military and dirty assets in my portfolio. I care a lot about sustainable development.'

From natural language understanding of the text we may try to figure out the following:
- **Holdings**: Where are we starting from? A blank slate? Or does the user have existing holdings? (*investor is starting from a blank portfolio and would like to invest $30000, portfolio construction problem instead of portfolio rebalancing*)
- **Benchmark**: If we're matching the properties of something, what are we matching? (*Aggressive, Moderate, or Conservative benchmark, Aggressive benchmark in this case as investor is 34 and investing for retirement (RRSP) - 30+ years till retirement, plus young person*)
- **Objective**: What are we trying to do? Are we minimizing or maximizing some property? Or are we matching a target? (*minimizing tracking error with respect to the benchmark, we can incorporate multiple objective functions later*)
- **Preferences**: What are the users unique preferences and constraints that we need to take into account when performing this calculation? Is there anything that can't be included? Do we need to have a specific weight at the end of our calculation? (*as investor wants highly diversified portfolio weights of each asset should be less than 5-10%, no military of fossil fuel assets in an optimal portfolio, high weight of sustainability assets, high weight of IT assets as investor works in IT*)

#### Concepts
This function uses the natural language understanding object to analyze the provided text for the top 10 concepts in it:

In [15]:
response = nlu.analyze(text=investor_text, features=Features(concepts=ConceptsOptions(limit=10)))
print(json.dumps(response.result, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 266,
    "features": 1
  },
  "language": "en",
  "concepts": [
    {
      "text": "Information technology",
      "relevance": 0.932431,
      "dbpedia_resource": "http://dbpedia.org/resource/Information_technology"
    },
    {
      "text": "Sustainability",
      "relevance": 0.80139,
      "dbpedia_resource": "http://dbpedia.org/resource/Sustainability"
    },
    {
      "text": "Investment",
      "relevance": 0.759159,
      "dbpedia_resource": "http://dbpedia.org/resource/Investment"
    },
    {
      "text": "Sustainable development",
      "relevance": 0.627417,
      "dbpedia_resource": "http://dbpedia.org/resource/Sustainable_development"
    },
    {
      "text": "Modern portfolio theory",
      "relevance": 0.598437,
      "dbpedia_resource": "http://dbpedia.org/resource/Modern_portfolio_theory"
    }
  ]
}


#### Keywords
This function uses the natural language understanding object to analyze the provided text for keywords in it:

In [16]:
response = nlu.analyze(text=investor_text, features=Features(keywords=KeywordsOptions(emotion=True, sentiment=True)))
print(json.dumps(response.result, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 266,
    "features": 1
  },
  "language": "en",
  "keywords": [
    {
      "text": "sustainable development",
      "sentiment": {
        "score": 0.837703,
        "label": "positive"
      },
      "relevance": 0.772288,
      "emotion": {
        "sadness": 0.347864,
        "joy": 0.257401,
        "fear": 0.046263,
        "disgust": 0.030665,
        "anger": 0.084685
      },
      "count": 1
    },
    {
      "text": "information technology sector",
      "sentiment": {
        "score": 0.586502,
        "label": "positive"
      },
      "relevance": 0.734161,
      "emotion": {
        "sadness": 0.123741,
        "joy": 0.120668,
        "fear": 0.101067,
        "disgust": 0.023275,
        "anger": 0.044114
      },
      "count": 1
    },
    {
      "text": "name",
      "sentiment": {
        "score": 0,
        "label": "neutral"
      },
      "relevance": 0.666938,
      "emotion": {
        "sadness": 0.1

#### Categories
This function uses the natural language understanding object to analyze the provided text for the categories that it fits into:

In [18]:
response = nlu.analyze(text=investor_text, features=Features(categories=CategoriesOptions()))
print(json.dumps(response.result, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 266,
    "features": 1
  },
  "language": "en",
  "categories": [
    {
      "score": 0.978387,
      "label": "/finance/investing"
    },
    {
      "score": 0.978387,
      "label": "/finance/investing/beginning investing"
    },
    {
      "score": 0.813729,
      "label": "/finance/investing/funds/mutual funds"
    }
  ]
}


#### Emotions
This function uses the natural language understanding object to analyze the provided text to find the emotion that it is conveying:

In [21]:
response = nlu.analyze(text=investor_text, features=Features(emotion=EmotionOptions(targets=['information technology','military'])))
print(json.dumps(response.result, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 266,
    "features": 1
  },
  "language": "en",
  "emotion": {
    "targets": [
      {
        "text": "information technology",
        "emotion": {
          "sadness": 0.123741,
          "joy": 0.120668,
          "fear": 0.101067,
          "disgust": 0.023275,
          "anger": 0.044114
        }
      },
      {
        "text": "military",
        "emotion": {
          "sadness": 0.468756,
          "joy": 0.027264,
          "fear": 0.136384,
          "disgust": 0.448297,
          "anger": 0.063849
        }
      }
    ],
    "document": {
      "emotion": {
        "sadness": 0.176835,
        "joy": 0.156152,
        "fear": 0.089034,
        "disgust": 0.108168,
        "anger": 0.066237
      }
    }
  }
}


#### Entities
This function uses the natural language understanding object to analyze the provided text to find relevant entities in the text:

In [24]:
all_feat = nlu.analyze(text=investor_text, features=Features(entities=EntitiesOptions(sentiment=True)))
all_feat.result

{'entities': [{'count': 1,
   'relevance': 0.951948,
   'sentiment': {'label': 'neutral', 'score': 0.0},
   'text': '$30000',
   'type': 'Quantity'}],
 'language': 'en',
 'usage': {'features': 1, 'text_characters': 266, 'text_units': 1}}

In [26]:
for i in range(len(all_feat.result['entities'])):
       print(all_feat.result['entities'][i]['text'],all_feat.result['entities'][i]['type'])

$30000 Quantity


In [27]:
for i in range(len(all_feat.result['entities'])):
    print(all_feat.result['entities'][i]['text'],all_feat.result['entities'][i]['type'])
    if all_feat.result['entities'][i]['type'] == 'Quantity':
        Cash = float( str(all_feat.result['entities'][0]['text']).replace('$',''))
        print(Cash)

$30000 Quantity
30000.0


#### Relations
Recognize when two entities are related, and identify the type of relation:

In [34]:
def getRelations(text):
    response=nlu.analyze(
        text=text,
        features=Features(relations=RelationsOptions())
    )
    print('Raw Results: ')
    print(json.dumps(response.result, indent=2))

In [35]:
getRelations(investor_text)

Raw Results: 
{
  "usage": {
    "text_units": 1,
    "text_characters": 266,
    "features": 1
  },
  "relations": [
    {
      "type": "hasAttribute",
      "sentence": "I am 34.",
      "score": 0.969477,
      "arguments": [
        {
          "text": "I",
          "location": [
            17,
            18
          ],
          "entities": [
            {
              "type": "Person",
              "text": "my"
            }
          ]
        },
        {
          "text": "34",
          "location": [
            22,
            24
          ],
          "entities": [
            {
              "type": "Age",
              "text": "34"
            }
          ]
        }
      ]
    }
  ],
  "language": "en"
}


#### Semantic Roles
This function uses the natural language understanding object to analyze the provided text to find relevant semantic roles in the text:

In [38]:
def getSemanticRoles(text):
    response=nlu.analyze(
        text=text,
        features=Features(semantic_roles=SemanticRolesOptions(entities=True,keywords=True,limit=50))
    )
    print('Raw Results: ')
    print(json.dumps(response.result, indent=2))

In [39]:
getSemanticRoles(investor_text)

Raw Results: 
{
  "usage": {
    "text_units": 1,
    "text_characters": 266,
    "features": 1
  },
  "semantic_roles": [
    {
      "subject": {
        "text": "I"
      },
      "sentence": " I am 34.",
      "object": {
        "text": "34"
      },
      "action": {
        "verb": {
          "text": "be",
          "tense": "present"
        },
        "text": "am",
        "normalized": "be"
      }
    },
    {
      "subject": {
        "text": "I"
      },
      "sentence": " I work information technology sector.",
      "object": {
        "text": "information technology sector",
        "keywords": [
          {
            "text": "information technology sector"
          }
        ],
        "entities": []
      },
      "action": {
        "verb": {
          "text": "work",
          "tense": "present"
        },
        "text": "work",
        "normalized": "work"
      }
    },
    {
      "subject": {
        "text": "I"
      },
      "sentence": " I would like t

# Step 3 - Inputs: Holdings, Benchmarks, Objectives, and Preferences

As with any optimization problem, there are a few things we need to determine before we get started:
- **Holdings**: Where are we starting from? A blank slate? Or does the user have existing holdings?
- **Benchmark**: If we're matching the properties of something, what are we matching?
- **Objective**: What are we trying to do? Are we minimizing or maximizing some property? Or are we matching a target?
- **Preferences**: What are the users unique preferences and constraints that we need to take into account when performing this calculation? Is there anything that can't be included? Do we need to have a specific weight at the end of our calculation?

In [40]:
#Initiliaze the request
optimization = {
    "portfolios": [],
    "objectives": [],
    "constraints": []
}

## Holdings

First we need to decide if we're starting from scratch, or if the user already has a portfolio of assets. We only need to specify the ID of the asset (our data set that we loaded above uses ticker) and the quantity in # of units. 

We also need to define how much cash - in addition to the assets in the portfolio - the client wants to add to this analysis.

We define this as an array of json objects:

In [41]:
##### PUT IN YOUR HOLDINGS HERE #####
Holdings = []
Holdings.append({"asset":"CX_US0533321024_NYQ","quantity":14}) #Example of a holding
Holdings.append({"asset":"CX_US0584981064_NYQ","quantity":162})
Holdings.append({"asset":"CX_US1696561059_NYQ","quantity":67})
Holdings.append({"asset":"CX_US1912161007_NYQ","quantity":68})
Holdings.append({"asset":"CX_US29379VAY92_USD","quantity":67})
Holdings.append({"asset":"CX_US30231GAN25_USD","quantity":68})
Holdings.append({"asset":"CX_US46120E6023_NSQ","quantity":13})
Holdings.append({"asset":"CX_US4878361082_NYQ","quantity":67})
Holdings.append({"asset":"CX_US5486611073_NYQ","quantity":67})
Holdings.append({"asset":"CX_US56585AAG76_USD","quantity":67})
Holdings.append({"asset":"CX_US651639AN69_USD","quantity":67})
Holdings.append({"asset":"CX_US70450Y1038_NSQ","quantity":67})
Holdings.append({"asset":"CX_US9100471096_NYQ","quantity":67})

#Example cash infusion
#Cash = 10000
Cash = 0
#####################################

#Initilialize the 'tradeable universe' - note that position units should be populated where assets are held.
tradeable_universe = {
    "name": "Universe",
    "type":"root",
    "holdings":[]
}
for index, row in universe_data.iterrows():
    holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
    if holding:
        tradeable_universe["holdings"].append(holding[0])
    else:
        tradeable_universe["holdings"].append({"asset":row["Ticker"],"quantity":0})

optimization["portfolios"].append(tradeable_universe)
# Debug
#print(json.dumps(optimization, indent=4, sort_keys=True))
#print("Holdings: " + str(Holdings))
#print("Cash: " + str(Cash))

## Benchmark

Next we need to pick a benchmark portfolio for our analysis. If we're simply optimizing to minimize variance/maximize return objectively, then this step can be skipped.

Instead, in our analysis we assume that our advisor is recommending an "off-the-shelf" portfolio that meets our risk and return objectives that we want to use, with some caveats. Here we select that benchmark by assigning the name to the Benchmark variable below.

Choices are: 
- Conservative
- Moderate
- Aggressive

In [42]:
#ENTER DETAILS HERE
Benchmark_name = "Aggressive"

#Assemble the benchmark portfolio using the above details:
Benchmark_col_name = "Benchmark - " + str(Benchmark_name)
benchmark = {
    "name": Benchmark_name,
    "type": "benchmark",
    "holdings": []
}
Benchmark_holdings = universe_data.filter(items=["Ticker","Last Close Price",Benchmark_col_name])[universe_data[Benchmark_col_name] != 0]
for index,row in Benchmark_holdings.iterrows():
    #Determine weight of each asset
    shares = row[Benchmark_col_name]
    holding = {"asset":row["Ticker"],"quantity":shares}
    benchmark["holdings"].append(holding)
    
optimization["portfolios"].append(benchmark)
#Debug
#print(Benchmark_holdings)
print(benchmark)

{'name': 'Aggressive', 'type': 'benchmark', 'holdings': [{'asset': 'CX_US4642872000_NYQ', 'quantity': 121.2807245}, {'asset': 'CX_US4642876142_NYQ', 'quantity': 164.3385374}, {'asset': 'CX_US4642878049_NYQ', 'quantity': 282.92544910000004}, {'asset': 'CX_US4642885135_NYQ', 'quantity': 225.73363430000003}, {'asset': 'CX_US46434G1031_NYQ', 'quantity': 191.46084630000001}]}


## Objective

Now we need to describe the overall mission of this analysis. We're going to hard-code this to be a minimization of variance of the returns of the portfolio - the standard [Markowitz approach](https://www.math.ust.hk/~maykwok/courses/ma362/07F/markowitz_JF.pdf).

For this type of analysis, the only parameters we need to be concerned with are:
- Whether or not we're trying to match the variance of returns relative to the benchmark, or simply minimizing it.
- The time step, which is contingent on the data set (in our case it's 30 days)

In [43]:
optimization["objectives"] = [{
       "sense": "minimize",
       "measure": "variance",
       "attribute": "return",
       "portfolio": "Universe",
       "TargetPortfolio": Benchmark_name,
       "timestep": 30,
       "description": "minimize tracking error squared (variance of the difference between Universe portfolio and Benchmark returns) at time 30 days"
}]

#Debug 
print(optimization["objectives"])

[{'sense': 'minimize', 'measure': 'variance', 'attribute': 'return', 'portfolio': 'Universe', 'TargetPortfolio': 'Aggressive', 'timestep': 30, 'description': 'minimize tracking error squared (variance of the difference between Universe portfolio and Benchmark returns) at time 30 days'}]


## Preferences

Now we get to the real meat of the analysis. This is where we can specify our custom constraints that will make this a _personalized_ analysis, resulting a portfolio with [as close to] identical properties (returns) as the benchmark portfolio, but with assets that meet the criteria specified below.

There's a lot of flexibility in how one would specify these constraints, but for the sake of this walkthrough, we're identifying three main types:

1. __Filtering__: Ensuring that a certain type of investment is not present in the portfolio (e.g. "sin stocks", military investments)
2. __ESG and Socially Responsible Investing Requirements__: Determine a minimum threshold for types of companies one would want included in a portfolio (e.g. I want my portfolio's average Governance score to be "High")
3. __Investment Allocation__: Determine specific weight requirements of the portfolio (e.g. 70% bonds, 30% stocks)

The user can also specify standard result requirements including:
- Allow/Prevent short-selling (e.g. weights can or can't go negative)
- Maximum allocation to any one security

In [44]:
##### CONSTRAINTS #####
#Uncomment the constraints you wish to apply:

#Filtering Constraints - "The final portfolio..."
Filtering_Constraints = []
#Filtering_Constraints.append("Has Tobacco")
#Filtering_Constraints.append("Has Alcohol")
#Filtering_Constraints.append("Has Gambling")
#Filtering_Constraints.append("Has Military") #Ex) No military investments desired in the portfolio
Filtering_Constraints.append("Has Fossil Fuels")
    
#ESG Constraints - Define both a field and an mean score desired (e.g. "Low","Average","High")
ESG_Constraints = []
#ESG_Constraints.append({"field":"Controversy","mean_score":"High"})
ESG_Constraints.append({"field":"Environmental","mean_score":"High"})
#ESG_Constraints.append({"field":"Governance","mean_score":"High"})
#ESG_Constraints.append({"field":"Social","mean_score":"High"})
#ESG_Constraints.append({"field":"Sustainability","mean_score":"High"})
                       
#Investment Allocation Constraints - I want to allocate [allocation]% of my portfoilio to be [value] identified by the [field] of the assets.
#inequality can be "greater-or-equal","less-or-equal" or "equal"
Allocation_Constraints = []
#Allocation_Constraints.append({"field":"ActiveOrPassive","value":"Passive","allocation":.1,"inequality":"greater-or-equal"}) #Field values are "Active" or "Passive"
#Allocation_Constraints.append({"field":"Geography","value":"International","allocation":.1,"inequality":"less-or-equal"}) #Field values are "Domestic" or "International"
#Allocation_Constraints.append({"field":"Asset Class","value":"Equity","allocation":.1,"inequality":"equal"}) #Field values are "Equity","Corporate Bonds","Municipal Bonds","Mortages","Mixed","Money Market", or "Commodities"
#Allocation_Constraints.append({"field":"Sector","value":"Information Technology","allocation":.1,"inequality":"greater-or-equal"}) #Field values are..
#"Consumer Discretionary","Consumer Staples","Energy","Financials","Health Care","Industrials","Information Technology","Materials","Real Estate","Utilities","Commodity","Government", or "Diversified"
Allocation_Constraints.append({"field":"Sector","value":"Industrials","allocation":.3,"inequality":"less-or-equal"}) #Field values are..

#Result requirements
AllowShortSales = False           #No short-selling
MaximumInvestmentWeight = .2      #20%
#MaximumNumberofPositions = 5     #Cardinality constraint on the portfolio

#Debug
print("Filtering Constraints: " + str(Filtering_Constraints))
print("ESG Constraints: " + str(ESG_Constraints))
print("Allocation Constraints: " + str(Allocation_Constraints))

Filtering Constraints: ['Has Fossil Fuels']
ESG Constraints: [{'field': 'Environmental', 'mean_score': 'High'}]
Allocation Constraints: [{'field': 'Sector', 'value': 'Industrials', 'allocation': 0.3, 'inequality': 'less-or-equal'}]


In [45]:
#FILTERING CONTRAINTS
#Add subportfolio (how the optimization algorithm knows which asset has which property)
for f in Filtering_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name":f,
        "type":"subportfolio",
        "holdings":[]
    }
    
    #Find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",f])[universe_data[f] != 0]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(holding[0])
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":f,
        "InPortfolio":"Universe",
        "relation":"equal",
        "constant":0.0,
        "description":"Excluding all securities which have the property " + f + "."
    })
    
#Debug
print(json.dumps(subportfolio, indent=4, sort_keys=True))

{
    "ParentPortfolio": "Universe",
    "holdings": [
        {
            "asset": "CX_US30231GAN25_USD",
            "quantity": 68
        },
        {
            "asset": "CX_US4642874659_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642875078_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642875987_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642876142_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642878049_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642882819_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642886463_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642886877_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US56585AAG76_USD",
            "quantity": 67
        },
        {
            "asset": "C

In [46]:
#ESG CONSTRAINTS
#Add subportfolio (how the optimization algorithm knows which asset has which property)
for e in ESG_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name": e["mean_score"] + e["field"],
        "type":"subportfolio",
        "holdings":[]
    }
    
    #Find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",e["field"]])[universe_data[e["field"]] == e["mean_score"]]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(holding[0])
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":e["mean_score"] + e["field"],
        "InPortfolio":"Universe",
        "relation":"greater-or-equal",
        "constant":.5,
        "description":"Creating an average " + e["field"] + " score of " + e["mean_score"] + "."
    })
    
#Debug
print(json.dumps(subportfolio, indent=4, sort_keys=True))

{
    "ParentPortfolio": "Universe",
    "holdings": [
        {
            "asset": "CX_US03232PAD06_USD",
            "quantity": 0
        },
        {
            "asset": "CX_US0533321024_NYQ",
            "quantity": 14
        },
        {
            "asset": "CX_US1696561059_NYQ",
            "quantity": 67
        },
        {
            "asset": "CX_US4642871762_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642872000_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642874402_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642874576_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642884146_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642885135_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642886463_NYQ",
            "quantity": 0
        },
        {
            "asset": "C

In [47]:
#ALLOCATION CONSTRAINTS
#Add subportfolio (how the optimization algorithm knows which asset has which property)
for a in Allocation_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name": a["value"],
        "type":"subportfolio",
        "holdings":[]
    }
    
    #Find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",a["field"]])[universe_data[a["field"]] == a["value"]]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(holding[0])
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":a["value"],
        "InPortfolio":"Universe",
        "relation":a["inequality"],
        "constant":a["allocation"],
        "description":"Weight of " + a["value"] + " in the portfolio should be " + a["inequality"] + " to " + str(a["allocation"]) + "."
    })
    
#Debug
print(json.dumps(subportfolio, indent=4, sort_keys=True))

{
    "ParentPortfolio": "Universe",
    "holdings": [
        {
            "asset": "CX_IE00BY7QL619_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US651639AN69_USD",
            "quantity": 67
        },
        {
            "asset": "CX_US9100471096_NYQ",
            "quantity": 67
        }
    ],
    "name": "Industrials",
    "type": "subportfolio"
}


In [48]:
#RESULT REQUIREMENTS

#No short-sale restriction
try:
    if AllowShortSales == False:
        optimization["constraints"].append({
           "attribute":"weight",
           "relation":"greater-or-equal",
           "members":"Universe",
           "constant":0,
           "description":"no short-sales for assets in Universe portfolio" 
        })
except:
    print("Short sales allowed")

#Maximum individual investment weight
try:
    optimization["constraints"].append({
       "attribute":"weight",
       "relation":"less-or-equal",
       "members":"Universe",
       "constant":MaximumInvestmentWeight,
       "description":"Weight of each asset from the Universe portfolio does not exceed " + str(MaximumInvestmentWeight*100) + "%."
    })
except:
    print("No maximum investment weight.")

#Maximum number of trades/positions
try:
    optimization["constraints"].append({
        "attribute": "count", 
        "relation": "less-or-equal", 
        "constant": MaximumNumberofPositions })
except:
    print("No maximum number of positions.")

#Minimum number of trades/positions
try:
    optimization["constraints"].append({
        "attribute": "count", 
        "relation": "greater-or-equal", 
        "constant": MinimumNumberofPositions })
except:
    print("No minimum number of positions.")

#Cash infusions
try:
    optimization["constraints"].append({
        "attribute:": "value",
        "portfolio": "Universe",
        "cashadjust": Cash,
        "description": "cash inflow of " + str(Cash) +" monetary units to the Universe portfolio"})
except:
    print("No cash infusions.")
    
# Debug
print(json.dumps(optimization["constraints"], indent=4, sort_keys=True))

No maximum number of positions.
No minimum number of positions.
[
    {
        "InPortfolio": "Universe",
        "attribute": "weight",
        "constant": 0.0,
        "description": "Excluding all securities which have the property Has Fossil Fuels.",
        "portfolio": "Has Fossil Fuels",
        "relation": "equal"
    },
    {
        "InPortfolio": "Universe",
        "attribute": "weight",
        "constant": 0.5,
        "description": "Creating an average Environmental score of High.",
        "portfolio": "HighEnvironmental",
        "relation": "greater-or-equal"
    },
    {
        "InPortfolio": "Universe",
        "attribute": "weight",
        "constant": 0.3,
        "description": "Weight of Industrials in the portfolio should be less-or-equal to 0.3.",
        "portfolio": "Industrials",
        "relation": "less-or-equal"
    },
    {
        "attribute": "weight",
        "constant": 0,
        "description": "no short-sales for assets in Universe portfolio",
 

## The Assembled Request:

We've assembled our payload! Below is what it looks like:

In [49]:
print(json.dumps(optimization, indent=4, sort_keys=True))

{
    "constraints": [
        {
            "InPortfolio": "Universe",
            "attribute": "weight",
            "constant": 0.0,
            "description": "Excluding all securities which have the property Has Fossil Fuels.",
            "portfolio": "Has Fossil Fuels",
            "relation": "equal"
        },
        {
            "InPortfolio": "Universe",
            "attribute": "weight",
            "constant": 0.5,
            "description": "Creating an average Environmental score of High.",
            "portfolio": "HighEnvironmental",
            "relation": "greater-or-equal"
        },
        {
            "InPortfolio": "Universe",
            "attribute": "weight",
            "constant": 0.3,
            "description": "Weight of Industrials in the portfolio should be less-or-equal to 0.3.",
            "portfolio": "Industrials",
            "relation": "less-or-equal"
        },
        {
            "attribute": "weight",
            "constant": 0,
          

# Step 4 - Submit the Request

Now that we have the payload assembled, we issue a [requests](http://docs.python-requests.org/en/master/) command to the Portfolio Optimization service running on the cloud. You'll want to provision your own copy of the application [which can be done here on Bluemix](https://console.bluemix.net/catalog/services/portfolio-optimization?env_id=ibm:yp:us-south). Fill in your uri and API credentials below before issuing the command.

In [50]:
##### ENTER YOUR PORTFOLIO OPTIMZIATION SERVICE CREDENTIALS #####
uri = "fss-analytics.mybluemix.net"
api_key = "7e8dcfd7e2fbe6661636a8b3a479a5491546172f8d8c0fd6eabc5e837b66c0fb8b54a6604835e9e49685b29d5a509918b068809af2879963294e8c6b7eefcbd345147370d4ee7829b8abf68f07712b27efe998baae301a2ac45e613097bd1e0e8a94e5847a047ead8468f92802ee90cbb3603d1327591a27922ac1dcea75ef4f"
#################################################################

url = "https://" + uri + "/api/v1/optimization/portfolio/construct"
headers = {'content-type': 'application/json', 'accept': 'application/json', 'X-IBM-Access-Token': api_key}
data = json.dumps(optimization)

import requests
r = requests.post(url,data=data,headers=headers)

print(r.text)


{
 "Holdings": [
  {
   "Asset": "CX_IE00BY7QL619_NYQ",
   "Quantity": 0,
   "OptimizedTrade": 95.7813100202294,
   "OptimizedQuantity": 95.7813100202294
  },
  {
   "Asset": "CX_US02079KAA51_USD",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "CX_US031162BG42_USD",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "CX_US031162BK53_USD",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "CX_US03232PAD06_USD",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "CX_US0533321024_NYQ",
   "Quantity": 14,
   "OptimizedTrade": 10.6575166550357,
   "OptimizedQuantity": 24.6575166550357
  },
  {
   "Asset": "CX_US0584981064_NYQ",
   "Quantity": 162,
   "OptimizedTrade": -127.528282992308,
   "OptimizedQuantity": 34.47171700769201
  },
  {
   "Asset": "CX_US1696561059_NYQ",
   "Quantity": 67,
   "OptimizedTrade": -67,
   "OptimizedQuantit

# Step 5 - Viewing your Results

So let's take a look at the results of our optimization calculation:

In [51]:
#Initialize Portfolio and field names
new_portfolio = []
fields = ["Name","Last Close Price","One Month Return"]

#Gather fields
for f in Filtering_Constraints:
    fields.append(f)
for e in ESG_Constraints:
    fields.append(e["field"])
for a in Allocation_Constraints:
    fields.append(a["field"])
    
#Assemble the data frame
for i in json.loads(r.text)["Holdings"]:
    if i["OptimizedQuantity"] != 0:
        security_data = universe_data.filter(items=fields)[universe_data["Ticker"] == i["Asset"]]
        security_data = security_data.values.tolist()[0]
        security_data.append(i["OptimizedQuantity"] * security_data[1])
        total_data = (str(i["Asset"]),float(i["OptimizedQuantity"])) + tuple(security_data)
        new_portfolio.append(total_data)
        
fields = ["Ticker","Quantity"]+fields+["Total Value"]
    
#Debug
#print(fields)
#print(new_portfolio)

In [52]:
OptimizedPortfolio = pd.DataFrame(new_portfolio, columns=fields)
OptimizedPortfolio

,Ticker,Quantity,Name,Last Close Price,One Month Return,Has Fossil Fuels,Environmental,Sector,Total Value
0,CX_IE00BY7QL619_NYQ,95.781310,JOHNSON CONTROLS INTERNATIONAL PLC,44.00,0.007803,0,Average,Industrials,4214.377641
1,CX_US0533321024_NYQ,24.657517,AUTOZONE INC,495.69,-0.012556,0,High,Consumer Discretionary,12222.484431
2,CX_US0584981064_NYQ,34.471717,BALL CORP,42.42,-0.011617,0,Average,Materials,1462.290235
3,CX_US29379VAY92_USD,155.301201,ENTERPRISE PRODUCTS OPERATING LLC,99.53,0.005616,0,Low,Energy,15457.128560
4,CX_US4642872000_NYQ,80.861433,iShares Core S&P 500,247.36,0.007253,0,High,Diversified,20001.884000
5,CX_US4642885135_NYQ,200.681056,iShares iBoxx $ High Yield Corporate Bond ETF,88.60,0.000763,0,High,Diversified,17780.341569
6,CX_US5486611073_NYQ,124.744793,LOWES COMPANIES INC,75.96,0.005499,0,Average,Consumer Discretionary,9475.614482
7,CX_US70450Y1038_NSQ,185.512544,PAYPAL HOLDINGS INC,58.96,0.019552,0,Low,Information Technology,10937.819602
8,CX_US9100471096_NYQ,107.192389,UNITED CONTINENTAL HOLDINGS INC,78.90,0.017505,0,Low,Industrials,8457.479480


## How close did we match the risk of the benchmark?

In [53]:
print("Tracking error between returns of the benchmark and the optimized portfolio is: %.2f%%" % 
      (100*(json.loads(r.text)["Metadata"]["ObjectiveValue"])**0.5))

Tracking error between returns of the benchmark and the optimized portfolio is: 1.91%


## Checking that the constraints are met:

In [54]:
#Sum up quantity * price
portfolio_value = 0
for j in new_portfolio:
    portfolio_value += j[-1]
print("Total Portfolio Value: " + str(portfolio_value))

#We use a pandas dataframe to do our aggregation analysis:
for f in fields[5:-1]:
    print("Aggregation: " + f)
    elements = set(OptimizedPortfolio[f])
    for e in elements:
        aggr = OptimizedPortfolio.filter(items=fields)[OptimizedPortfolio[f] == e]
        aggr_pct = aggr["Total Value"].sum() / portfolio_value
        print("  Total allocation to " + str(e) + " is " + str(aggr_pct*100) + "%.")

Total Portfolio Value: 100009.41999999997
Aggregation: Has Fossil Fuels
  Total allocation to 0 is 100.0%.
Aggregation: Environmental
  Total allocation to Low is 34.84914485225837%.
  Total allocation to High is 49.99999999999997%.
  Total allocation to Average is 15.150855147741671%.
Aggregation: Sector
  Total allocation to Energy is 15.455672635415803%.
  Total allocation to Information Technology is 10.936789356197808%.
  Total allocation to Industrials is 12.670663544554428%.
  Total allocation to Diversified is 37.778666818850986%.
  Total allocation to Consumer Discretionary is 21.69605514428026%.
  Total allocation to Materials is 1.4621525007007294%.


In [55]:
OptimizedPortfolio['Weight'] = OptimizedPortfolio.apply(lambda row: row['Quantity'] * row['Last Close Price'] / portfolio_value, axis=1)
OptimizedPortfolio

,Ticker,Quantity,Name,Last Close Price,One Month Return,Has Fossil Fuels,Environmental,Sector,Total Value,Weight
0,CX_IE00BY7QL619_NYQ,95.781310,JOHNSON CONTROLS INTERNATIONAL PLC,44.00,0.007803,0,Average,Industrials,4214.377641,0.042140
1,CX_US0533321024_NYQ,24.657517,AUTOZONE INC,495.69,-0.012556,0,High,Consumer Discretionary,12222.484431,0.122213
2,CX_US0584981064_NYQ,34.471717,BALL CORP,42.42,-0.011617,0,Average,Materials,1462.290235,0.014622
3,CX_US29379VAY92_USD,155.301201,ENTERPRISE PRODUCTS OPERATING LLC,99.53,0.005616,0,Low,Energy,15457.128560,0.154557
4,CX_US4642872000_NYQ,80.861433,iShares Core S&P 500,247.36,0.007253,0,High,Diversified,20001.884000,0.200000
5,CX_US4642885135_NYQ,200.681056,iShares iBoxx $ High Yield Corporate Bond ETF,88.60,0.000763,0,High,Diversified,17780.341569,0.177787
6,CX_US5486611073_NYQ,124.744793,LOWES COMPANIES INC,75.96,0.005499,0,Average,Consumer Discretionary,9475.614482,0.094747
7,CX_US70450Y1038_NSQ,185.512544,PAYPAL HOLDINGS INC,58.96,0.019552,0,Low,Information Technology,10937.819602,0.109368
8,CX_US9100471096_NYQ,107.192389,UNITED CONTINENTAL HOLDINGS INC,78.90,0.017505,0,Low,Industrials,8457.479480,0.084567


In [56]:
OptimizedPortfolio.groupby("Environmental")["Weight"].sum()

Environmental
Average    0.151509
High       0.500000
Low        0.348491
Name: Weight, dtype: float64

In [57]:
OptimizedPortfolio.groupby("Sector")["Weight"].sum()

Sector
Consumer Discretionary    0.216961
Diversified               0.377787
Energy                    0.154557
Industrials               0.126707
Information Technology    0.109368
Materials                 0.014622
Name: Weight, dtype: float64

In [58]:
OptimizedPortfolio.groupby("Has Fossil Fuels")["Weight"].sum()

Has Fossil Fuels
0    1.0
Name: Weight, dtype: float64

## Compute portfolio expected return:

In [59]:
OptimizedPortfolio['Return'] = OptimizedPortfolio.apply(lambda row: row['One Month Return'] * row['Weight'], axis=1)

In [60]:
print("Portfolio Return: %.5f%%" % (100*OptimizedPortfolio["Return"].sum()))

Portfolio Return: 0.52185%


In [61]:
print("Annualized Portfolio Return: %.5f%%" % (100*12*OptimizedPortfolio["Return"].sum()))

Annualized Portfolio Return: 6.26219%


## Compute portfolio variance and tracking error:

In [62]:
import numpy as np

Read covariance matrix

In [63]:
df_cov = pd.read_csv('Covar_Universe_Data.csv', sep=',', header=None)
cov_matr = df_cov.values

Get holdings of initial and optimal portfolios as well as the benchmark portfolio

In [64]:
x_opt = []
x_init = []
for i in json.loads(r.text)["Holdings"]:
    x_opt.append(float(i["OptimizedQuantity"]))
    x_init.append(float(i["Quantity"]))
x_opt = np.array(x_opt)
x_init = np.array(x_init)

In [65]:
x_bench = universe_data[Benchmark_col_name].values

Compute portfolio values

In [66]:
val = universe_data["Last Close Price"].values
ret = universe_data["One Month Return"].values

In [67]:
portf_init_val = np.dot(val.T, x_init) # Initial portfolio value
portf_init_val

100009.41999999998

In [68]:
portf_opt_val = np.dot(val.T, x_opt) # Optimal portfolio value
portf_opt_val

100009.41999999997

In [69]:
np.allclose(portf_opt_val, OptimizedPortfolio["Total Value"].sum()) # Sanity check

True

Compute portfolio expected return, standard deviation and tracking error

In [70]:
w_opt   = x_opt * val / portf_opt_val
w_init  = x_init * val / portf_opt_val
w_bench = x_bench * val / portf_opt_val

In [71]:
ret_opt  = np.dot(ret.T, w_opt)
ret_init = np.dot(ret.T, w_init)

In [72]:
print("Expected monthly return of optimal portfolio: % .5f%%" % (100*ret_opt))
print("Expected monthly return of initial portfolio: % .5f%%" % (100*ret_init))

Expected monthly return of optimal portfolio:  0.52185%
Expected monthly return of initial portfolio: -0.13031%


In [73]:
print("Expected annual return of optimal portfolio: % .5f%%" % (12*100*ret_opt))
print("Expected annual return of initial portfolio: % .5f%%" % (12*100*ret_init))

Expected annual return of optimal portfolio:  6.26219%
Expected annual return of initial portfolio: -1.56377%


In [74]:
std_opt  = np.dot(np.dot(w_opt.T, cov_matr), w_opt)**0.5
std_init = np.dot(np.dot(w_init.T, cov_matr), w_init)**0.5

In [75]:
print("Standard deviation of optimal portfolio: %.8f" % std_opt)
print("Standard deviation of initial portfolio: %.8f" % std_init)

Standard deviation of optimal portfolio: 0.03026262
Standard deviation of initial portfolio: 0.02941583


In [76]:
tr_err_opt  = np.dot(np.dot((w_opt-w_bench).T, cov_matr), (w_opt-w_bench))**0.5
tr_err_init = np.dot(np.dot((w_init-w_bench).T, cov_matr), (w_init-w_bench))**0.5

In [77]:
print("Tracking error of optimal portfolio: %.8f" % tr_err_opt)
print("Tracking error of initial portfolio: %.8f" % tr_err_init)

Tracking error of optimal portfolio: 0.01910889
Tracking error of initial portfolio: 0.03596899


In [78]:
print("Number of assets in optimal portfolio: %d" % np.count_nonzero(w_opt))
print("Number of assets in initial portfolio: %d" % np.count_nonzero(w_init))

Number of assets in optimal portfolio: 9
Number of assets in initial portfolio: 13
